In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

np.random.seed(42)

In [2]:
# Reading h state data from excel sheet
z = pd.read_excel("Link 2 Measurements.xlsx")

# Convert Dataframe to numpy array
z = z.to_numpy()

In [3]:
# Initialization

# Ai => cross-section of Tank i (cm^2)
A1, A2, A3, A4 = 28, 32, 28, 32

# ai => cross-section of outlet hole of Tank i (cm^2)
a1, a2, a3, a4 = 0.071, 0.057, 0.071, 0.057

# proportionality constants
kc = 0.5 # % (V/cm)
g = 981 # % (cm/s^2)
gamma1, gamma2 = 0.7, 0.6
k1, k2 = 3.33, 3.35 # % cm^3/Vs

# initial h
h0 = np.array([12.4, 12.7, 1.8, 1.4])

# voltages
v1, v2 = [3, 3] # % V

# No. of states
n=4
# No. of particles
N=10000
total_data_length = 10000
C = np.array([[kc, 0, 0, 0],
              [0, kc, 0, 0]])

ts = 0.1

### Prior

In [4]:
p0 = pow(10,5)*np.eye(4)
x0 = h0.reshape(4,1)

L = np.linalg.cholesky(p0)
x0 = (x0 @ np.ones([1, N])).T + (np.random.rand(N, n) @ L)/N

# Initialization of the states
x1_post = x0[:, 0]
x2_post = x0[:, 1]
x3_post = x0[:, 2]
x4_post = x0[:, 3]

# Process Noise
Q = 0.1 * np.eye(4) # process noise
w = np.linalg.cholesky(Q) @ np.random.rand(n, N)/N # roughening of the prior
w1 = w[0, :]
w2 = w[1, :]
w3 = w[2, :]
w4 = w[3, :]

# Prior roughening
x1_post += w1.T
x2_post += w2.T
x3_post += w3.T
x4_post += w4.T

In [5]:
x1_prior = np.zeros(N)
x2_prior = np.zeros(N)
x3_prior = np.zeros(N)
x4_prior = np.zeros(N)
x1_post_data, x2_post_data, x3_post_data, x4_post_data = np.zeros(total_data_length), np.zeros(total_data_length), np.zeros(total_data_length), np.zeros(total_data_length)

### Particle Filter

In [ ]:
X_post = []

for k in tqdm(range(len(z))):
# for k in tqdm(range(1000)):
    
    # Prediction Step
    x1_prior = x1_post + ts*((-a1/A1)*np.sqrt(2*g*x1_post) + (a3/A1)*np.sqrt(2*g*x3_post) + (gamma1*k1*v1)/A1 + w1)
    x2_prior = x2_post + ts*((-a2/A2)*np.sqrt(2*g*x2_post) + (a4/A2)*np.sqrt(2*g*x4_post) + (gamma2*k1*v2)/A2 + w2)
    x3_prior = x3_post + ts*((-a3/A3)*np.sqrt(2*g*x3_post) + ((1 - gamma2)*k2*v1)/A3 + w3)
    x4_prior = x4_post + ts*((-a4/A4)*np.sqrt(2*g*x4_post) + ((1 - gamma1)*k1*v2)/A4 + w4)
    x1_prior = abs(x1_prior)
    x2_prior = abs(x2_prior)
    x3_prior = abs(x3_prior)
    x4_prior = abs(x4_prior)
    x_prior = np.array([x1_prior, x2_prior, x3_prior, x4_prior]).T
    
    # Likelihood Step
    # Importance Weights
    z_true = np.array(C @ z[k]).reshape(2,1) @ np.ones([1,N])
    R = 1 * np.eye(2)
    z_est = C @ x_prior.T
    v = z_true - z_est
    q = [np.exp(-0.5 * (v[:, i].T @ np.linalg.inv(R) @ v[:, i])) for i in range(N)]
    
    # Normalize the weights
    wt = [q[i]/sum(q) for i in range(N)]
    
    # Selection
    # Resampling
    M = len(wt)
    Q = np.cumsum(wt)
    indx = np.zeros(N)
    T = np.linspace(0, 1 - (1/N), N) + np.random.rand(N)/N
    i = 0
    j = 0
    while(i < N and j < M):
        while(Q[j] < T[i]):
            j += 1
        indx[i] = j
        x1_post[i] = x1_prior[j]
        x2_post[i] = x2_prior[j]
        x3_post[i] = x3_prior[j]
        x4_post[i] = x4_prior[j]
        i += 1    
    X_post.append([x1_post.mean(), x2_post.mean(), x3_post.mean(), x4_post.mean()])

  0%|          | 23/10001 [01:30<11:00:48,  3.97s/it]

In [ ]:
def plot_X(X, h, label):
    # Extracting X
    X_1 = [x[0] for x in X]
    X_2 = [x[1] for x in X]
    X_3 = [x[2] for x in X]
    X_4 = [x[3] for x in X]


    fig, ax = plt.subplots(2,2, figsize=(12, 8))
    ax[0][0].plot(X_1)
    ax[0][1].plot(X_2)
    ax[1][0].plot(X_3)
    ax[1][1].plot(X_4)

    if type(h) == np.ndarray:
        ax[0][0].plot(h[:,0])
        ax[0][1].plot(h[:,1])
        ax[1][0].plot(h[:,2])
        ax[1][1].plot(h[:,3])
    
    ax[0][0].set_xlabel("k (Time Steps)")
    ax[0][1].set_xlabel("k (Time Steps)")
    ax[1][0].set_xlabel("k (Time Steps)")
    ax[1][1].set_xlabel("k (Time Steps)")
    ax[0][0].set_ylabel(label)
    ax[0][1].set_ylabel(label)
    ax[1][0].set_ylabel(label)
    ax[1][1].set_ylabel(label)

    ax[0][0].legend([label + "1", "Measured h1 Value from CSV"])
    ax[0][1].legend([label + "2", "Measured h2 Value from CSV"])
    ax[1][0].legend([label + "3", "Measured h3 Value from CSV"])
    ax[1][1].legend([label + "4", "Measured h4 Value from CSV"])

    fig.suptitle(label)
    plt.show()

In [ ]:
# Plotting X posterior
plot_X(X_post, z[:len(X_post)], 'X_Posterior')

In [ ]:
X_post[-1]

[12.286329555610704,
 12.715180427659662,
 1.6377875202192915,
 1.4140518676187503]